In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
# from datasets import load_metric
# !pip install evaluate
# 
from sklearn.model_selection import train_test_split
import csv
from optuna import Trial
from typing import Dict, Union, Any
import evaluate

In [2]:
df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/Ghadhab/dataset.csv', index_col=0, encoding='utf_8_sig')
# df.fillna('', inplace=True)
label2id={'Adaptive':0, 'Corrective':1, 'Perfective':2}
# df = df.rename(columns={'labels':'label','msgs':'message','diffs':'diff'})
df = df.replace({"labels": label2id})
# df = df['diffs']
# # print(df)
df
# test_sample = df.sample(n=3, random_state=1)

/tmp/ipykernel_1482/2424605049.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"labels": label2id})


,repo,commit,labels,msgs,diffs,feature
user,,,,,,
ponsonio,RxJava,0531b8bff5c14d9504beefb4ad47f473e3a22932,2,Change hasException to hasThrowable--,diff --git a/rxjava-core/src/main/java/rx/Noti...,"[1, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0950c46beda335819928585f1262dfe1dca78a0b,0,Trying to extend the Scheduler interface accor...,diff --git a/rxjava-core/src/main/java/rx/Sche...,"[2, 44, 0, 0, 30, 0, 0, 1, 18, 0, 0, 0, 0, 0, ..."
ponsonio,RxJava,0f92fdd8e6422d5b79c610a7fd8409d222315a49,0,RunAsync method for outputting multiple values--,diff --git a/rxjava-contrib/rxjava-async-util/...,"[2, 53, 0, 0, 42, 0, 0, 1, 45, 1, 0, 0, 0, 0, ..."
ponsonio,RxJava,100f571c9a2835d5a30a55374b9be74c147e031f,1,forEach with Action1 but not Observer--I re-re...,diff --git a/language-adaptors/rxjava-groovy/s...,"[1, 5, 122, 9, 10, 9, 4, 1, 5, 18, 2, 0, 0, 0,..."
ponsonio,RxJava,191f023cf5253ea90647bc091dcaf55ccdce81cc,1,1.x: Fix Completable swallows- OnErrorNotImple...,diff --git a/src/main/java/rx/Completable.java...,"[1, 1, 0, 0, 0, 0, 0, 1, 21, 0, 0, 0, 0, 0, 0,..."
...,...,...,...,...,...,...
jenkinsci,clearcase-plugin,51e9da224f80254476a7dc446bca817b505381d8,2,Use a temporary file to decrease memory consum...,diff --git a/src/main/java/hudson/plugins/clea...,"[2, 12, 0, 4, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,..."
jexp,batch-import,609d6c4b1eea2c33d9fb950fcbb9ba9dc1f80fc3,2,added a more memory efficient structure for st...,diff --git a/src/main/java/org/neo4j/batchimpo...,"[10, 159, 29, 35, 9, 2, 1, 5, 106, 0, 4, 8, 0,..."
hdiv,hdiv,19b650c78a1c76f4fd90274d7f163f863c0d39e4,2,Memory and performance optimizations,diff --git a/hdiv-config/src/main/java/org/hdi...,"[31, 302, 131, 140, 170, 89, 53, 7, 88, 14, 17..."


In [3]:
# from datasets import Dataset, load_metric

In [4]:
train, temp_df = train_test_split(df, test_size=0.3, random_state=42)
val, test = train_test_split(temp_df, test_size=0.5, random_state=42)

In [5]:
len(train)

1246

In [6]:
train['labels'].value_counts()

labels
1    424
0    420
2    402
Name: count, dtype: int64

In [7]:
test['labels'].value_counts()

labels
2    100
1     89
0     79
Name: count, dtype: int64

In [8]:
# id2label = {0: "Adaptive", 1: "POSITIVE"}
# label2id = {"NEGATIVE": 0, "POSITIVE": 1}

In [9]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained(
    "/root/autodl-tmp/CommitFit/models/codebert-base", num_labels=3
)
tokenizer = AutoTokenizer.from_pretrained('/root/autodl-tmp/CommitFit/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/CommitFit/models/codebert-base were not used when initializing RobertaForSequenceClassification: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at /root/autodl-tmp/CommitFit/models/codebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probab

In [10]:
training_args = TrainingArguments(
    output_dir="./my_awesome_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    report_to="none"
)

In [11]:
encoded_train = tokenizer(train['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_test = tokenizer(test['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)
encoded_val = tokenizer(val['diffs'].astype(str).to_list(), return_tensors='pt',truncation=True, padding=True)

In [12]:
class CommitDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [13]:
train_dataset = CommitDataset(encoded_train, list(train['labels']))
test_dataset = CommitDataset(encoded_test, list(test['labels']))
val_dataset = CommitDataset(encoded_val, list(val['labels']))

In [14]:
from sklearn import  metrics
def compute_metrics(eval_pred):
    accuracy = evaluate.load('../../evaluate/metrics/accuracy')
    precision =  evaluate.load("../../evaluate/metrics/precision")
    recall =  evaluate.load("../../evaluate/metrics/recall")
    f1 =  evaluate.load("../../evaluate/metrics/f1")
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = accuracy.compute(predictions=predictions, references=labels)
    precision = precision.compute(predictions=predictions, references=labels, average='macro')
    recall = recall.compute(predictions=predictions, references=labels,average='macro')
    f1 = f1.compute(predictions=predictions, references=labels,average='macro')
    # accuracy_score = metrics.accuracy_score(labels,predictions)
    # precision_score = metrics.precision_score(labels,predictions, average='weighted')
    # recall_score = metrics.recall_score(labels,predictions,average='weighted')
    # f1_score = metrics.f1_score(labels,predictions,average='weighted')
    # 混淆矩阵
    cm = metrics.confusion_matrix(labels, predictions)
    print("Confusion Matrix:\n", cm)
    print(metrics.classification_report(labels, predictions, digits=4))
    # return {"classification_report": classification_report}
    # return {"precision": precision_score,"recall": recall_score, "f1":f1_score, "accuracy": accuracy_score}
    return {"precision": precision, "recall": recall, "f1": f1, "accuracy": accuracy}

In [15]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    # data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/tmp/ipykernel_1482/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.934900,{'precision': 0.5312209761307699},{'recall': 0.5278799228237431},{'f1': 0.5277462846090297},{'accuracy': 0.5280898876404494}
2,No log,0.891761,{'precision': 0.6254333696273792},{'recall': 0.5889550940112738},{'f1': 0.5816536739613662},{'accuracy': 0.5880149812734082}
3,No log,0.878111,{'precision': 0.5990460157126823},{'recall': 0.5952530927250028},{'f1': 0.5879091668565352},{'accuracy': 0.5955056179775281}
4,No log,0.829695,{'precision': 0.6674504303825316},{'recall': 0.6296939431770893},{'f1': 0.6262906904416338},{'accuracy': 0.6292134831460674}
5,No log,0.828614,{'precision': 0.650456579583938},{'recall': 0.6370559527862899},{'f1': 0.6363535588676926},{'accuracy': 0.6367041198501873}
6,No log,0.904459,{'precision': 0.6323606970878239},{'recall': 0.6291397117239814},{'f1': 0.6298965199275982},{'accuracy': 0.6292134831460674}
7,0.773700,0.976373,{'precision': 0.6294224294224294},{'recall': 0.6255192373169901},{'f1': 0.6260873280355465},{'accuracy': 0.6254681647940075}
8,0.773700,1.049900,{'precision': 0.6220833333333333},{'recall': 0.6223205841183369},{'f1': 0.6199029068594286},{'accuracy': 0.6217228464419475}


Confusion Matrix:
 [[51 21 17]
 [27 48 15]
 [18 28 42]]
              precision    recall  f1-score   support

           0     0.5312    0.5730    0.5514        89
           1     0.4948    0.5333    0.5134        90
           2     0.5676    0.4773    0.5185        88

    accuracy                         0.5281       267
   macro avg     0.5312    0.5279    0.5277       267
weighted avg     0.5309    0.5281    0.5277       267



/tmp/ipykernel_1482/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[38 17 34]
 [ 8 49 33]
 [ 5 13 70]]
              precision    recall  f1-score   support

           0     0.7451    0.4270    0.5429        89
           1     0.6203    0.5444    0.5799        90
           2     0.5109    0.7955    0.6222        88

    accuracy                         0.5880       267
   macro avg     0.6254    0.5890    0.5817       267
weighted avg     0.6258    0.5880    0.5815       267



...

/tmp/ipykernel_1482/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[59 17 13]
 [25 54 11]
 [15 19 54]]
              precision    recall  f1-score   support

           0     0.5960    0.6629    0.6277        89
           1     0.6000    0.6000    0.6000        90
           2     0.6923    0.6136    0.6506        88

    accuracy                         0.6255       267
   macro avg     0.6294    0.6255    0.6261       267
weighted avg     0.6291    0.6255    0.6259       267



/tmp/ipykernel_1482/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[59 12 18]
 [25 47 18]
 [12 16 60]]
              precision    recall  f1-score   support

           0     0.6146    0.6629    0.6378        89
           1     0.6267    0.5222    0.5697        90
           2     0.6250    0.6818    0.6522        88

    accuracy                         0.6217       267
   macro avg     0.6221    0.6223    0.6199       267
weighted avg     0.6221    0.6217    0.6196       267



TrainOutput(global_step=624, training_loss=0.6867468295953213, metrics={'train_runtime': 154.9799, 'train_samples_per_second': 80.398, 'train_steps_per_second': 5.033, 'total_flos': 2622714547912704.0, 'train_loss': 0.6867468295953213, 'epoch': 8.0})

In [16]:
fewshot_metrics = trainer.evaluate(eval_dataset=test_dataset)
fewshot_metrics

/tmp/ipykernel_1482/77294352.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Confusion Matrix:
 [[39 20 20]
 [12 66 11]
 [ 5 31 64]]
              precision    recall  f1-score   support

           0     0.6964    0.4937    0.5778        79
           1     0.5641    0.7416    0.6408        89
           2     0.6737    0.6400    0.6564       100

    accuracy                         0.6306       268
   macro avg     0.6447    0.6251    0.6250       268
weighted avg     0.6440    0.6306    0.6280       268



{'eval_loss': 0.8756884932518005,
 'eval_precision': {'precision': 0.6447384486858171},
 'eval_recall': {'recall': 0.6250813065946049},
 'eval_f1': {'f1': 0.6249882444057201},
 'eval_accuracy': {'accuracy': 0.6305970149253731},
 'eval_runtime': 1.3101,
 'eval_samples_per_second': 204.563,
 'eval_steps_per_second': 12.976,
 'epoch': 8.0}

In [17]:
#发送多种类型的邮件
from email.mime.multipart import MIMEMultipart
import smtplib

from email.mime.text import MIMEText
msg_from = '915803745@qq.com'  # 发送方邮箱
passwd = 'vcuosuurrgkfbdai'   #就是上面的授权码
 
# to= ['g.zhang@gotion.com', 'j.tong@gotion.com'] #接受方邮箱
to= ['j.tong@gotion.com'] #接受方邮箱
#设置邮件内容
#MIMEMultipart类可以放任何内容
msg = MIMEMultipart()
conntent=f"{fewshot_metrics}"
#把内容加进去
msg.attach(MIMEText(conntent,'plain','utf-8'))
 
#设置邮件主题
msg['Subject']="强化学习模型训练完毕"
 
#发送方信息
msg['From']=msg_from
 
#开始发送
 
#通过SSL方式发送，服务器地址和端口
s = smtplib.SMTP_SSL("smtp.qq.com", 465)
# 登录邮箱
s.login(msg_from, passwd)
#开始发送
s.sendmail(msg_from,to,msg.as_string())
print("强化学习模型训练完毕")

强化学习模型训练完毕
